In [9]:
#pip install sentence_transformers

In [7]:
#!pip install chromadb

     ---------------------------------------- 0.0/83.5 kB ? eta -:--:--
     ---------------------------------------  81.9/83.5 kB 2.3 MB/s eta 0:00:01
     ---------------------------------------- 83.5/83.5 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     ---------------------------------------- 45.9/45.9 kB 2.4 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
 

In [62]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
import numpy as np
import os

In [63]:
def text_embedding(text) -> None:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(text)

In [64]:
# examples, how embedding works
embed = text_embedding("hi how are you")

ls = embed.tolist()

In [65]:
ls

[-0.036796290427446365,
 0.022186167538166046,
 0.0842624306678772,
 0.03755715489387512,
 -0.03824450820684433,
 -0.0675714984536171,
 0.04600771144032478,
 -0.0011718309251591563,
 -0.06845701485872269,
 0.029643841087818146,
 -0.040180932730436325,
 0.022064145654439926,
 -0.018984666094183922,
 -0.042301349341869354,
 0.04092909395694733,
 0.013511068187654018,
 0.08018608391284943,
 -0.08453015238046646,
 -0.13972586393356323,
 0.0019706375896930695,
 -0.019132371991872787,
 0.02252325788140297,
 -0.02307927794754505,
 0.046114932745695114,
 -0.060092926025390625,
 -0.04144846275448799,
 -0.008367613889276981,
 0.009640536271035671,
 0.011113584972918034,
 -0.049456048756837845,
 -0.011552104726433754,
 0.022797053679823875,
 -0.04451688006520271,
 -0.004326390568166971,
 -0.015570363961160183,
 -0.020759250968694687,
 -0.015429015271365643,
 -0.1517704725265503,
 -0.008485314436256886,
 -0.007988626137375832,
 -0.005390317644923925,
 -0.04691177234053612,
 -0.005756687838584185,


In [66]:
def generate_context(query):
    vector=text_embedding(query).tolist()  # list of all vectors of my query.
    
    results=collection.query(    
        query_embeddings=vector,
        n_results=15,
        include=["documents"]
    )
    
    res = "\n".join(str(item) for item in results['documents'][0])
    return res

In [67]:
def chat_completion(system_prompt, user_prompt,length=1000):
    final_prompt=f"""<s>[INST]<<SYS>>
    {system_prompt}
    <</SYS>>
    
    {user_prompt} [/INST]"""
    return client.text_generation(prompt=final_prompt,max_new_tokens = length).strip()

In [68]:
df=pd.read_csv(r"C:\Users\hp\Downloads\oscars.csv")
df=df.loc[df['year_ceremony'] == 2023]
df=df.dropna(subset=['film'])
df.loc[:, 'category'] = df['category'].str.lower()
df.loc[:, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' to win the award'
df.loc[df['winner'] == False, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' but did not win' 

In [69]:
client = chromadb.Client()
collection = client.get_or_create_collection("oscars-2023")

In [70]:
docs=df["text"].tolist() 
ids= [str(x) for x in df.index.tolist()]
collection.add(
    documents=docs,
    ids=ids
)

Add of existing embedding ID: 10639
Add of existing embedding ID: 10640
Add of existing embedding ID: 10641
Add of existing embedding ID: 10642
Add of existing embedding ID: 10643
Add of existing embedding ID: 10644
Add of existing embedding ID: 10645
Add of existing embedding ID: 10646
Add of existing embedding ID: 10647
Add of existing embedding ID: 10648
Add of existing embedding ID: 10649
Add of existing embedding ID: 10650
Add of existing embedding ID: 10651
Add of existing embedding ID: 10652
Add of existing embedding ID: 10653
Add of existing embedding ID: 10654
Add of existing embedding ID: 10655
Add of existing embedding ID: 10656
Add of existing embedding ID: 10657
Add of existing embedding ID: 10658
Add of existing embedding ID: 10659
Add of existing embedding ID: 10660
Add of existing embedding ID: 10661
Add of existing embedding ID: 10662
Add of existing embedding ID: 10663
Add of existing embedding ID: 10664
Add of existing embedding ID: 10665
Add of existing embedding ID

In [78]:
from huggingface_hub import InferenceClient

In [93]:
URI=' http://127.0.0.1:8890'
client = InferenceClient(model=URI)

In [94]:
#query="What did Ke Huy Quan work on?"
#query="Which movie won the best music award?"
query="Did Lady Gaga win an award at Oscars 2023?"
#query="Who is the music director of RRR?"
context=generate_context(query)

In [95]:
context

'Monika Willi got nominated under the category, film editing, for the film Tár but did not win\nMary Zophres got nominated under the category, costume design, for the film Babylon but did not win\nMusic and Lyric by Lady Gaga and BloodPop got nominated under the category, music (original song), for the film Top Gun: Maverick but did not win\nMark Coulier, Jason Baird and Aldo Signoretti got nominated under the category, makeup and hairstyling, for the film Elvis but did not win\nSara Gunnarsdóttir and Pamela Ribon got nominated under the category, short film (animated), for the film My Year of Dicks but did not win\nDede Gardner, Jeremy Kleiner and Frances McDormand, Producers got nominated under the category, best picture, for the film Women Talking but did not win\nBaz Luhrmann, Catherine Martin, Gail Berman, Patrick McCormick and Schuyler Weiss, Producers got nominated under the category, best picture, for the film Elvis but did not win\nLaura Poitras, Howard Gertler, John Lyons, Na

In [96]:

system_prompt = """\
You are a helpful AI assistant that can answer questions on Oscar 2023 awards. Answer based on the context provided. 
If you cannot find the correct answerm, say I don't know. Be concise and just include the response.
"""

In [97]:
user_prompt=f"""
Based on the context:
{context}
Answer the below query:
{query}
"""

In [98]:
user_prompt

'\nBased on the context:\nMonika Willi got nominated under the category, film editing, for the film Tár but did not win\nMary Zophres got nominated under the category, costume design, for the film Babylon but did not win\nMusic and Lyric by Lady Gaga and BloodPop got nominated under the category, music (original song), for the film Top Gun: Maverick but did not win\nMark Coulier, Jason Baird and Aldo Signoretti got nominated under the category, makeup and hairstyling, for the film Elvis but did not win\nSara Gunnarsdóttir and Pamela Ribon got nominated under the category, short film (animated), for the film My Year of Dicks but did not win\nDede Gardner, Jeremy Kleiner and Frances McDormand, Producers got nominated under the category, best picture, for the film Women Talking but did not win\nBaz Luhrmann, Catherine Martin, Gail Berman, Patrick McCormick and Schuyler Weiss, Producers got nominated under the category, best picture, for the film Elvis but did not win\nLaura Poitras, Howar

In [99]:
# Llama2
chat_completion(system_prompt,user_prompt)

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/%20http://127.0.0.1:8890 (Request ID: Oz0yK-OcPyXO32e62Hkv4)